In [1]:
import os
import time
from brainscore_vision import score
from brainscore_vision.benchmarks.majajhong2015 import *
from brainscore_vision.benchmarks.majajhong2015.benchmark import load_assembly
from brainscore_vision import benchmark_registry

from brainscore_vision.metrics.internal_consistency.ceiling import (
    _SplitHalvesConsistency, SpearmanBrownCorrection, XarrayDefaults, CrossValidationSingle
)

benchmark_registry

/n/home02/alvarez/.conda/envs/brainscore/lib/python3.11/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


{'MajajHong2015.V4-pls': <function brainscore_vision.benchmarks.majajhong2015.benchmark.DicarloMajajHong2015V4PLS()>,
 'MajajHong2015.IT-pls': <function brainscore_vision.benchmarks.majajhong2015.benchmark.DicarloMajajHong2015ITPLS()>,
 'MajajHong2015public.V4-pls': <function brainscore_vision.benchmarks.majajhong2015.benchmark.MajajHongV4PublicBenchmark()>,
 'MajajHong2015public.IT-pls': <function brainscore_vision.benchmarks.majajhong2015.benchmark.MajajHongITPublicBenchmark()>,
 'MajajHong2015public.V4-temporal-pls': <function brainscore_vision.benchmarks.majajhong2015.<lambda>()>,
 'MajajHong2015public.IT-temporal-pls': <function brainscore_vision.benchmarks.majajhong2015.<lambda>()>}

In [2]:
import cupy as cp
from scipy.stats import t as t_dist
from pdb import set_trace

def pearsonr_cupy(x, y):
    # Ensure inputs are CuPy arrays
    x = cp.asarray(x.data)
    y = cp.asarray(y.data)
    
    # Check that the lengths match
    if x.size != y.size:
        raise ValueError("Input arrays must have the same length.")

    # Number of observations
    n = x.size

    # Compute means
    mean_x = cp.mean(x)
    mean_y = cp.mean(y)

    # Compute deviations from the mean
    x_dev = x - mean_x
    y_dev = y - mean_y

    # Compute numerator (covariance) and denominator (standard deviations)
    cov = cp.sum(x_dev * y_dev)
    std_x = cp.sqrt(cp.sum(x_dev ** 2))
    std_y = cp.sqrt(cp.sum(y_dev ** 2))

    # Pearson correlation coefficient
    r = cov / (std_x * std_y)

    # Compute the t-statistic
    df = n - 2  # degrees of freedom
    t_stat = r * cp.sqrt(df / ((1 - r ** 2) + 1e-9))  # Add small value to avoid division by zero

    # Use SciPy to compute p-value based on the t-distribution CDF
    p_value = 2 * t_dist.cdf(-abs(cp.asnumpy(t_stat)), df)  # Convert to NumPy for p-value computation

    return cp.asnumpy(r), p_value

In [3]:
import scipy
import numpy as np 
import pandas as pd

from brainscore_core.metrics import Metric, Score
from brainio.assemblies import walk_coords, DataAssembly
from brainscore_vision.metric_helpers.xarray_utils import XarrayCorrelation

from brainscore_vision.metrics.internal_consistency.ceiling import (
    _SplitHalvesConsistency, SpearmanBrownCorrection, XarrayDefaults, CrossValidationSingle
)

class PearsonCorrelation:
    """
    Computes the Pearson r between two halves of an assembly.
    """

    def __init__(self, stimulus_coord=XarrayDefaults.stimulus_coord,
                 neuroid_dim=XarrayDefaults.neuroid_dim, neuroid_coord=XarrayDefaults.neuroid_coord):
        # correlation = scipy.stats.pearsonr
        correlation = lambda x,y: (cp.corrcoef(x.data, y.data)[0,1].item(), None)
        self._correlation = XarrayCorrelation(correlation, correlation_coord=stimulus_coord,
                                              neuroid_coord=neuroid_coord)
        self._neuroid_dim = neuroid_dim

    def __call__(self, half1, half2):
        return self._correlation(half1, half2)

    def aggregate(self, scores):
        return scores.median(dim=self._neuroid_dim)
    
class SplitHalfWrapper:
        def __init__(self, split_coord, consistency_metric: Metric, correction):
            self._split_coord = split_coord
            self._consistency_metric = consistency_metric
            self._correction = correction

        def __call__(self, half1, half2):
            half1, half2 = self._average_repetitions(half1), self._average_repetitions(half2)
            consistency = self._consistency_metric(half1, half2)
            corrected_consistency = self._correction(consistency, n=2)
            return corrected_consistency

        def _average_repetitions(self, assembly):
            repetition_dims = assembly[self._split_coord].dims
            nonrepetition_coords = [coord for coord, dims, values in walk_coords(assembly)
                                    if dims == repetition_dims and coord != self._split_coord]
            average = assembly.multi_groupby(nonrepetition_coords).mean(dim=repetition_dims)
            return average
        
def _average_repetitions(assembly, split_coord):
    repetition_dims = assembly[split_coord].dims
    nonrepetition_coords = [coord for coord, dims, values in walk_coords(assembly)
                            if dims == repetition_dims and coord != split_coord]
    average = assembly.multi_groupby(nonrepetition_coords).mean(dim=repetition_dims)
    return average        

In [4]:
from scipy.stats import pearsonr

def xarray_correlation(prediction, target, neuroid_coord, correlation_coord, correlation=pearsonr):
    # align
    prediction = prediction.sortby([correlation_coord, neuroid_coord])
    target = target.sortby([correlation_coord, neuroid_coord])
    
    # Assert alignment
    assert np.array(prediction[correlation_coord].values == target[correlation_coord].values).all()
    assert np.array(prediction[neuroid_coord].values == target[neuroid_coord].values).all()

    # Compute correlation per neuroid
    neuroid_dims = target[neuroid_coord].dims
    assert len(neuroid_dims) == 1
    correlations = []
    
    for i, coord_value in enumerate(target[neuroid_coord].values):
        target_neuroids = target.isel(**{neuroid_dims[0]: i})  # `isel` is about 10x faster than `sel`
        prediction_neuroids = prediction.isel(**{neuroid_dims[0]: i})
        r, p = correlation(target_neuroids, prediction_neuroids)
        correlations.append(r)

    # Package the result
    result = Score(correlations,
                   coords={coord: (dims, values)
                           for coord, dims, values in walk_coords(target) if dims == neuroid_dims},
                   dims=neuroid_dims)
    
    return result

def xarray_correlation_cp(prediction, target, neuroid_coord, correlation_coord, correlation=pearsonr):
    
    # align
    prediction = prediction.sortby([correlation_coord, neuroid_coord])
    target = target.sortby([correlation_coord, neuroid_coord])
    
    # Assert alignment
    assert np.array(prediction[correlation_coord].values == target[correlation_coord].values).all()
    assert np.array(prediction[neuroid_coord].values == target[neuroid_coord].values).all()

    # Compute correlation per neuroid
    neuroid_dims = target[neuroid_coord].dims
    assert len(neuroid_dims) == 1
    correlations = []
    
    for i, coord_value in enumerate(target[neuroid_coord].values):
        target_neuroids = target.isel(**{neuroid_dims[0]: i})  # `isel` is about 10x faster than `sel`
        prediction_neuroids = prediction.isel(**{neuroid_dims[0]: i})
        r, p = correlation(target_neuroids.data, prediction_neuroids.data)
        correlations.append(r)
        
    # Package the result
    result = Score(correlations,
                   coords={coord: (dims, values)
                           for coord, dims, values in walk_coords(target) if dims == neuroid_dims},
                   dims=neuroid_dims)
    
    return result

In [5]:
# load a subset of the data
region = "IT"
access = "public"
time_interval = None
assembly_repetition = load_assembly(average_repetitions=False, 
                                    region=region, 
                                    access=access, time_interval=time_interval)

# Looks like 148480 trials acros 168 neural sites
assembly_repetition.shape

(148480, 168)

In [6]:
import cupy as cp 
import xarray as xr

In [7]:
# Convert the underlying data to CuPy
data_cupy = assembly_repetition.map_blocks(cp.asarray)
data_cupy.shape

(148480, 168)

In [8]:
data_cupy.device

<CUDA Device 0>

In [9]:
print(f"DataArray type after conversion: {type(data_cupy.data)}")

DataArray type after conversion: <class 'cupy.cuda.memory.MemoryPointer'>


In [10]:
# Convert the underlying data to CuPy while keeping the Xarray structure
data_cupy = assembly_repetition.copy(deep=True)  # Optional: Copy to preserve the original
data_cupy.data = cp.asarray(data_cupy.data)  # Convert to CuPy

# Now, data_cupy is still an Xarray DataArray but using CuPy arrays under the hood
print(f"DataArray type after conversion: {type(data_cupy.data)}")
print(f"Is it still an Xarray DataArray? {isinstance(data_cupy, xr.DataArray)}")

DataArray type after conversion: <class 'cupy.ndarray'>
Is it still an Xarray DataArray? True


In [11]:
data_cupy

<xarray.NeuronRecordingAssembly 'dicarlo.MajajHong2015.public' (
                                                                presentation: 148480,
                                                                neuroid: 168)>
array([[ 0.06092933, -0.72559214,  0.42031842, ..., -0.18637353,
         0.28050217,  0.30726328],
       [-0.8479065 ,  0.89549875,  0.9519652 , ..., -2.260826  ,
        -1.1872205 ,  0.35321778],
       [-1.618728  , -0.7792393 , -1.3977592 , ...,  0.22751288,
        -0.17249185,  0.10181567],
       ...,
       [-0.83211136, -1.5217171 ,  1.1938343 , ...,  0.6354223 ,
         0.42337415, -2.2552547 ],
       [-0.83211136, -1.158598  , -0.17778037, ..., -1.6090839 ,
         0.0775381 ,  1.0052351 ],
       [-0.83211136, -0.7954789 , -0.8635877 , ...,  0.6354223 ,
        -0.614134  , -0.2989609 ]], dtype=float32)
Coordinates:
  * neuroid          (neuroid) MultiIndex
  - neuroid_id       (neuroid) object 'Chabo_L_M_5_9' ... 'Tito_L_M_8_0'
  - arr              (neuroid) object 'M' 'M' 'M' 'M' 'M' ... 'M' 'M' 'M' 'M'
  - col              (neuroid) int64 9 9 8 9 8 8 7 7 5 6 ... 1 0 1 0 1 0 0 1 1 0
  - hemisphere       (neuroid) object 'L' 'L' 'L' 'L' 'L' ... 'L' 'L' 'L' 'L'
  - subregion        (neuroid) object 'cIT' 'cIT' 'cIT' ... 'pIT' 'pIT' 'pIT'
  - animal           (neuroid) object 'Chabo' 'Chabo' 'Chabo' ... 'Tito' 'Tito'
  - y                (neuroid) float64 0.2 0.6 0.2 1.0 0.6 ... 1.0 1.0 1.8 1.4
  - x                (neuroid) float64 1.8 1.8 1.4 1.8 ... -1.8 -1.4 -1.4 -1.8
  - row              (neuroid) int64 5 6 5 7 6 7 9 7 9 8 ... 4 4 5 5 6 6 7 7 9 8
  * presentation     (presentation) MultiIndex
  - image_id         (presentation) object '8a72e2bfdb8c267b57232bf96f069374d...
  - repetition       (presentation) int64 0 18 18 18 18 18 ... 16 16 16 17 17 17
  - stimulus         (presentation) int64 0 426 427 428 429 ... 2569 2566 0 1 2
  - stimulus_id      (presentation) object '8a72e2bfdb8c267b57232bf96f069374d...
  - id               (presentation) int64 1 418 419 420 421 ... 3197 641 642 643
  - background_id    (presentation) object 'ecd40f3f6d7a4d6d88134d648884e0b9b...
  - s                (presentation) float64 1.0 1.0 1.0 ... 1.246 1.296 0.9114
  - image_file_name  (presentation) object 'astra_rx+00.000_ry+00.000_rz+00.0...
  - filename         (presentation) object 'astra_rx+00.000_ry+00.000_rz+00.0...
  - rxy              (presentation) float64 -0.0 -0.0 -0.0 ... 0.02724 -10.4
  - tz               (presentation) float64 0.0 0.0 0.0 ... -0.269 -0.599 0.211
  - category_name    (presentation) object 'Cars' 'Faces' ... 'Fruits' 'Fruits'
  - rxz_semantic     (presentation) float64 0.0 0.0 0.0 ... 13.22 -2.621 -14.72
  - ty               (presentation) float64 0.0 0.0 0.0 ... -0.191 -0.213 0.277
  - ryz              (presentation) float64 -0.0 -0.0 -0.0 ... -16.89 -0.2055
  - object_name      (presentation) object 'car_astra' 'face0' ... 'apple'
  - variation        (presentation) int64 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3
  - size             (presentation) float64 256.0 256.0 256.0 ... 256.0 256.0
  - rxy_semantic     (presentation) float64 90.0 -0.0 -0.0 ... 0.02724 -10.4
  - ryz_semantic     (presentation) float64 -0.0 -0.0 -0.0 ... -16.89 -0.2055
  - rxz              (presentation) float64 0.0 0.0 0.0 ... 13.22 -2.621 -14.72
    time_bin         object (70, 170)
    region           (neuroid) <U2 'IT' 'IT' 'IT' 'IT' ... 'IT' 'IT' 'IT' 'IT'
Attributes:
    stimulus_set_identifier:  hvm-public
    stimulus_set:                     id                             backgrou...
    identifier:               dicarlo.MajajHong2015.public

In [12]:
split_coord='repetition'
stimulus_coord = 'stimulus_id'
neuroid_dim = 'neuroid'
neuroid_coord = 'neuroid_id'
cross_validation_kwargs=None

consistency_metric = PearsonCorrelation(stimulus_coord=stimulus_coord, 
                                        neuroid_dim=neuroid_dim,
                                        neuroid_coord=neuroid_coord)
aggregate = consistency_metric.aggregate
    
correction = SpearmanBrownCorrection()
_consistency = SplitHalfWrapper(split_coord=split_coord,
                                consistency_metric=consistency_metric, correction=correction)
_aggregate = aggregate
cross_validation_defaults = dict(train_size=0.5, split_coord=split_coord,
                                 stratification_coord=None, unique_split_values=True)
cross_validation_kwargs = {**cross_validation_defaults, **(cross_validation_kwargs or {})}
_cross_validation = CrossValidationSingle(**cross_validation_kwargs)

In [13]:
# assembly_repetition.values, data_cupy.data.device

In [14]:
# result = _cross_validation(assembly_repetition, apply=_consistency, aggregate=_aggregate)
# result

In [15]:
result = _cross_validation(data_cupy, apply=_consistency, aggregate=_aggregate)
result

cross-validation: 100%|██████████| 10/10 [00:37<00:00,  3.72s/it]


<xarray.DataArray ()>
array(0.81579938)
Attributes:
    raw:      <xarray.DataArray (split: 10, neuroid: 168)>\narray([[0.1951266...
    error:    <xarray.DataArray ()>\narray(0.00144955)

In [24]:
# let's manually step through the CrossValidationSingle step
cross_validation_values, splits = _cross_validation._split.build_splits(data_cupy)
cross_validation_values.shape, splits

((51,),
 [(array([ 4, 34, 14, 31, 10, 28, 45, 35, 18, 20, 25,  6,  7, 48,  1, 16,  0,
          15,  5, 11,  9,  8, 12, 43, 37]),
   array([22,  2, 49, 26, 33, 44, 30, 50, 32, 27,  3, 29, 47, 41, 39, 21, 40,
          38, 19, 24, 13, 42, 17, 46, 36, 23])),
  (array([ 5, 27, 40, 37, 36,  3,  9, 22, 39, 47,  7, 23, 34, 25,  4,  0, 15,
          18, 41, 45, 16, 26, 12, 43, 49]),
   array([29, 20, 32, 28, 14, 17, 35, 11,  1, 13, 46, 33, 21, 24, 48, 31, 30,
          50, 42, 10,  2, 44,  8,  6, 19, 38])),
  (array([35, 17, 28, 23, 10, 41, 32, 18,  4, 42, 30, 20, 46, 43,  8, 13,  7,
          11, 49, 27, 12,  2, 50, 21,  6]),
   array([34,  3, 45,  5, 33, 40, 31, 39, 14, 44, 48, 29, 16, 26, 15, 22, 47,
          19,  9,  1, 38,  0, 37, 36, 24, 25])),
  (array([ 9, 30, 15, 27, 24, 34,  6, 43, 45, 14, 40, 19, 41, 21, 25, 32, 10,
          49, 33, 13, 37, 22, 48, 20, 42]),
   array([44, 12,  1, 35, 26, 11, 38, 36, 23,  3, 50,  0, 29, 16, 46, 18, 28,
          17, 31,  8, 47,  4,  5, 39,  7,  2]

In [127]:
from tqdm import tqdm
from brainio.transform import subset
from brainscore_vision.benchmark_helpers.neural_common import average_repetition
from brainscore_vision.metrics import Score
from scipy.stats import pearsonr

assembly = data_cupy
assembly = assembly_repetition
split_scores = []
for split_iterator, (train_indices, test_indices) \
        in tqdm(enumerate(splits), total=len(splits), desc='cross-validation'):
    train_values, test_values = cross_validation_values[train_indices], cross_validation_values[test_indices]
    train = subset(assembly, train_values, dims_must_match=False)
    #train_mask = np.isin(assembly.repetition.values, train_values)
    #train = assembly.values[train_mask,:]

    test = subset(assembly, test_values, dims_must_match=False)
    #test_mask = np.isin(assembly.repetition.values, test_values)
    #test = assembly.values[test_mask,:]
    break
#     split_score = _consistency(train, test)
#     # half1 = _average_repetitions(train, split_coord)
#     # half2 = _average_repetitions(test, split_coord)
    
#     split_score = split_score.expand_dims('split')
#     split_score['split'] = [split_iterator]
#     split_scores.append(split_score)
# split_scores = Score.merge(*split_scores)

cross-validation:   0%|          | 0/10 [00:00<?, ?it/s]


In [128]:
half1 = _average_repetitions(train, split_coord)
half2 = _average_repetitions(test, split_coord)

In [40]:
consistency_metric = PearsonCorrelation(stimulus_coord=stimulus_coord, 
                                        neuroid_dim=neuroid_dim,
                                        neuroid_coord=neuroid_coord)

In [ ]:
import time



In [129]:
start = time.time()
result = xarray_correlation(half1, half2, neuroid_coord=neuroid_coord, correlation_coord=stimulus_coord, 
                            correlation = pearsonr )
dur = time.time() - start
print(dur)
result

0.41633081436157227


<xarray.Score (neuroid: 168)>
array([0.10811096, 0.504932  , 0.53868685, 0.11657354, 0.25491836,
       0.35493351, 0.59296329, 0.65164218, 0.6586783 , 0.14006893,
       0.55154857, 0.39946343, 0.55939795, 0.10347588, 0.37699932,
       0.77089555, 0.69562897, 0.39067407, 0.49646249, 0.34069526,
       0.3424567 , 0.57717172, 0.91977458, 0.93242979, 0.68967423,
       0.70344605, 0.52193558, 0.48511821, 0.84695327, 0.79294521,
       0.88606576, 0.87770991, 0.90284587, 0.40013254, 0.37138418,
       0.87541615, 0.92044026, 0.93643258, 0.94025365, 0.84738742,
       0.32354753, 0.57157735, 0.0690411 , 0.79673518, 0.55605572,
       0.77922228, 0.77548262, 0.86279645, 0.86835826, 0.89391366,
       0.21148826, 0.27370082, 0.91642346, 0.81333157, 0.88258987,
       0.93092745, 0.90569964, 0.87998009, 0.75605968, 0.62207326,
       0.52075192, 0.23940529, 0.18871029, 0.27494836, 0.41677347,
       0.34639632, 0.28437451, 0.09097989, 0.57403019, 0.54195222,
       0.4629572 , 0.38961404, 0.42303977, 0.19303495, 0.64353029,
       0.77303222, 0.5903391 , 0.59129278, 0.26194876, 0.22508991,
       0.50438337, 0.77277545, 0.56142681, 0.11260438, 0.52192   ,
       0.75101688, 0.39788546, 0.31450444, 0.68128582, 0.6949521 ,
       0.28203167, 0.59866701, 0.47651402, 0.43817456, 0.476169  ,
       0.38348723, 0.74541021, 0.4572628 , 0.89182448, 0.87329795,
       0.71897638, 0.49393116, 0.8693364 , 0.71663906, 0.71314308,
       0.75781251, 0.6702976 , 0.60078081, 0.42029017, 0.88811335,
       0.62422602, 0.80500742, 0.65838536, 0.65790645, 0.59595006,
       0.62260806, 0.83059928, 0.76802377, 0.59226103, 0.59864485,
       0.65268128, 0.73890846, 0.6246776 , 0.79303515, 0.68371098,
       0.78002656, 0.69771685, 0.7944141 , 0.73040186, 0.8338887 ,
       0.73100292, 0.84223784, 0.73727245, 0.61219884, 0.6528912 ,
       0.82350534, 0.79873857, 0.81986827, 0.89689488, 0.80377644,
       0.90475744, 0.77312228, 0.89918618, 0.70798382, 0.87450466,
       0.70375441, 0.57334509, 0.71505607, 0.8072978 , 0.78417574,
       0.81867227, 0.83911139, 0.54753131, 0.74799843, 0.80802591,
       0.56361113, 0.76803807, 0.77070407, 0.67145957, 0.66229198,
       0.70502224, 0.73179694, 0.4968113 , 0.74740507, 0.77098315,
       0.80281609, 0.70476049, 0.73713681])
Coordinates:
  * neuroid     (neuroid) MultiIndex
  - neuroid_id  (neuroid) object 'Chabo_L_A_2_4' ... 'Tito_L_M_9_8'
  - arr         (neuroid) object 'A' 'A' 'A' 'A' 'A' 'A' ... 'M' 'M' 'M' 'M' 'M'
  - col         (neuroid) int64 4 3 5 0 1 2 3 4 5 6 2 ... 4 5 6 7 8 1 3 4 5 7 8
  - hemisphere  (neuroid) object 'L' 'L' 'L' 'L' 'L' 'L' ... 'L' 'L' 'L' 'L' 'L'
  - subregion   (neuroid) object 'cIT' 'cIT' 'aIT' 'cIT' ... 'pIT' 'pIT' 'pIT'
  - animal      (neuroid) object 'Chabo' 'Chabo' 'Chabo' ... 'Tito' 'Tito'
  - y           (neuroid) float64 -1.0 -0.6 -0.6 -0.2 -0.2 ... 1.8 1.8 1.8 1.8
  - x           (neuroid) float64 -0.2 -0.6 0.2 -1.8 -1.4 ... -0.2 0.2 1.0 1.4
  - row         (neuroid) int64 2 3 3 4 4 4 4 4 4 4 5 ... 8 8 8 8 8 9 9 9 9 9 9
  - region      (neuroid) object 'IT' 'IT' 'IT' 'IT' ... 'IT' 'IT' 'IT' 'IT'

In [130]:
.4 / .15

2.666666666666667

In [126]:
start = time.time()
result = xarray_correlation_cp(half1, half2, neuroid_coord=neuroid_coord, correlation_coord=stimulus_coord, 
                               correlation = lambda x,y: (cp.corrcoef(x, y)[0,1].item(), None) )
dur = time.time() - start
print(dur)
result

0.15602660179138184


<xarray.Score (neuroid: 168)>
array([0.10811096, 0.50493199, 0.53868688, 0.11657354, 0.25491836,
       0.35493351, 0.5929633 , 0.65164216, 0.65867823, 0.14006892,
       0.55154856, 0.3994634 , 0.55939798, 0.10347588, 0.37699931,
       0.77089552, 0.69562894, 0.39067406, 0.49646245, 0.34069525,
       0.34245669, 0.57717164, 0.91977454, 0.93242987, 0.68967419,
       0.70344607, 0.52193554, 0.48511819, 0.84695322, 0.79294522,
       0.88606571, 0.87770988, 0.90284595, 0.40013255, 0.37138416,
       0.87541613, 0.92044033, 0.93643253, 0.94025368, 0.84738744,
       0.32354755, 0.57157732, 0.0690411 , 0.79673524, 0.55605571,
       0.77922226, 0.77548257, 0.86279655, 0.86835822, 0.89391359,
       0.21148825, 0.2737008 , 0.91642347, 0.81333158, 0.88258985,
       0.93092737, 0.90569957, 0.87998007, 0.75605972, 0.62207329,
       0.52075192, 0.23940531, 0.1887103 , 0.27494839, 0.41677345,
       0.34639633, 0.28437449, 0.09097989, 0.57403022, 0.54195222,
       0.46295715, 0.38961403, 0.42303979, 0.19303494, 0.64353038,
       0.77303225, 0.59033915, 0.59129289, 0.26194877, 0.22508991,
       0.50438337, 0.77277552, 0.56142681, 0.11260437, 0.52192001,
       0.75101686, 0.39788549, 0.31450445, 0.68128587, 0.69495216,
       0.28203168, 0.59866698, 0.47651402, 0.43817456, 0.47616902,
       0.38348723, 0.74541022, 0.45726281, 0.89182443, 0.87329801,
       0.71897633, 0.49393115, 0.86933641, 0.71663906, 0.71314306,
       0.75781249, 0.67029757, 0.60078086, 0.42029012, 0.88811336,
       0.62422602, 0.80500736, 0.65838532, 0.65790647, 0.59595002,
       0.62260807, 0.83059934, 0.7680238 , 0.592261  , 0.59864482,
       0.65268122, 0.73890852, 0.62467758, 0.79303516, 0.68371105,
       0.78002651, 0.69771682, 0.79441416, 0.73040182, 0.83388864,
       0.7310029 , 0.8422378 , 0.73727247, 0.61219885, 0.65289118,
       0.82350528, 0.79873854, 0.81986828, 0.89689485, 0.80377651,
       0.90475742, 0.77312224, 0.89918604, 0.70798384, 0.87450458,
       0.70375436, 0.57334509, 0.71505607, 0.80729786, 0.78417574,
       0.81867224, 0.83911137, 0.54753132, 0.7479984 , 0.8080259 ,
       0.56361108, 0.768038  , 0.77070413, 0.67145957, 0.66229202,
       0.70502224, 0.73179695, 0.49681127, 0.74740505, 0.77098316,
       0.80281601, 0.70476054, 0.73713682])
Coordinates:
  * neuroid     (neuroid) MultiIndex
  - neuroid_id  (neuroid) object 'Chabo_L_A_2_4' ... 'Tito_L_M_9_8'
  - arr         (neuroid) object 'A' 'A' 'A' 'A' 'A' 'A' ... 'M' 'M' 'M' 'M' 'M'
  - col         (neuroid) int64 4 3 5 0 1 2 3 4 5 6 2 ... 4 5 6 7 8 1 3 4 5 7 8
  - hemisphere  (neuroid) object 'L' 'L' 'L' 'L' 'L' 'L' ... 'L' 'L' 'L' 'L' 'L'
  - subregion   (neuroid) object 'cIT' 'cIT' 'aIT' 'cIT' ... 'pIT' 'pIT' 'pIT'
  - animal      (neuroid) object 'Chabo' 'Chabo' 'Chabo' ... 'Tito' 'Tito'
  - y           (neuroid) float64 -1.0 -0.6 -0.6 -0.2 -0.2 ... 1.8 1.8 1.8 1.8
  - x           (neuroid) float64 -0.2 -0.6 0.2 -1.8 -1.4 ... -0.2 0.2 1.0 1.4
  - row         (neuroid) int64 2 3 3 4 4 4 4 4 4 4 5 ... 8 8 8 8 8 9 9 9 9 9 9
  - region      (neuroid) object 'IT' 'IT' 'IT' 'IT' ... 'IT' 'IT' 'IT' 'IT'

In [103]:
# consistency_metric(half1, half2)

In [123]:
cp.corrcoef(half1.data[:,0], half2.data[:,0])[0,1].item()

0.5560557118368319

In [92]:
half1.data.transpose().shape, half2.data.transpose().shape

((168, 3200), (168, 3200))

In [87]:
half1.data.transpose()

array([[-0.13490714,  0.1940739 ,  0.15228647, ...,  0.4535077 ,
         0.2627261 , -0.5260907 ],
       [-0.43063596,  0.31669247, -0.66462725, ...,  0.366535  ,
        -0.07539596, -0.106332  ],
       [-0.45261225,  0.5510013 , -0.35180023, ...,  0.612802  ,
        -0.6328119 , -0.42061844],
       ...,
       [ 0.240606  ,  0.39146066,  0.5492965 , ...,  0.41977802,
         0.11960585,  0.11060322],
       [ 0.13254511, -0.25623715,  0.54298735, ...,  0.50503   ,
         0.3500974 , -0.24538483],
       [ 0.17550433,  0.22261089,  0.01614855, ...,  0.31118324,
        -0.04281639, -0.15389812]], dtype=float32)